In [1]:
import os, sys
import argparse
import numpy as np
from collections import defaultdict
import json
import time
import multiprocessing
import copy
import os.path as osp
# from utils import IdGenerator, id2rgb
import pdb
import torch
try:
    import PIL.Image     as Image
except:
    print("Failed to import the image processing packages.")
    sys.exit(-1)
from pycocotools.coco import COCO
import numpy as np
# import skimage.io as io
import pylab

In [2]:
stage ='train'

In [3]:
inst_gt_json_file = "/mnt/data-disk2/xinting/project/dataset/LVIS/lvis_v0.5_"+stage+".json"
data_path = './dataset/LVIS/images/'+stage+'2017'
with open(inst_gt_json_file, 'r') as f:
    inst_gt = json.load(f)

In [4]:
import torch
import torchvision
min_keypoints_per_image = 10


def _count_visible_keypoints(anno):
    return sum(sum(1 for v in ann["keypoints"][2::3] if v > 0) for ann in anno)


def _has_only_empty_bbox(anno):
    return all(any(o <= 1 for o in obj["bbox"][2:]) for obj in anno)


def has_valid_annotation(anno):
    # if it's empty, there is no annotation
    if len(anno) == 0:
        return False
    # if all boxes have close to zero area, there is no annotation
    if _has_only_empty_bbox(anno):
        return False
    # keypoints task have a slight different critera for considering
    # if an annotation is valid
    if "keypoints" not in anno[0]:
        return True
    # for keypoint detection tasks, only consider valid images those
    # containing at least min_keypoints_per_image
    if _count_visible_keypoints(anno) >= min_keypoints_per_image:
        return True
    return False

class COCODataset(torchvision.datasets.coco.CocoDetection):
    def __init__(self, root, ann_file, sorted_id, remove_images_without_annotations=False):
        super(COCODataset, self).__init__(root, ann_file)
        self.ids = sorted(self.ids)

        # filter images without detection annotations
        if remove_images_without_annotations:
            ids = []
            for img_id in self.ids:
                ann_ids = self.coco.getAnnIds(imgIds=img_id, catIds = sorted_id,iscrowd=None)
#                 ann_ids = self.coco.getAnnIds(imgIds=img_id, iscrowd=None)

                anno = self.coco.loadAnns(ann_ids)
                if has_valid_annotation(anno):
                    ids.append(img_id)
            self.ids = ids

        self.categories = {cat['id']: cat['name'] for cat in self.coco.cats.values()}

        self.category_id_to_sorted_id = {
            v: i + 1 for i, v in enumerate(sorted_id)
        }
        self.sorted_id_to_category_id = {
            v: k for k, v in self.category_id_to_sorted_id.items()
        }
        self.id_to_img_map = {k: v for k, v in enumerate(self.ids)}
        self.img_map_to_id = {v: k for k, v in self.id_to_img_map.items()}

#         self._transforms = transforms
        
    def __getitem__(self, idx):
        img, anno = super(COCODataset, self).__getitem__(idx)
#         print(anno)
        return img, anno, idx
    
    def get_img_info(self, index):
        img_id = self.id_to_img_map[index]
        img_data = self.coco.imgs[img_id]
        return img_data



In [7]:
sorted_inst = sorted(inst_gt['categories'], key=lambda k: k['instance_count'], reverse=True) 
sorted_instance_count = [item['instance_count'] for item in sorted_inst]
sorted_class_name = [item['name'] for item in sorted_inst]
sorted_frequency = [item['frequency'] for item in sorted_inst]
sorted_img_count = [item['image_count'] for item in sorted_inst]
sorted_id=[]
for i in sorted_inst:
    sorted_id.append(i['id'])
    

In [8]:
coco_train = COCODataset(data_path, inst_gt_json_file, sorted_id, True)

loading annotations into memory...
Done (t=14.49s)
creating index...
index created!


In [11]:
import sys, math
repeat_thresh = 0.001
num_images = len(coco_train)
# 1. For each category c, compute the fraction of images that contain it: f(c)
category_freq = {}
for cls, num_img in enumerate(sorted_img_count):
    category_freq[cls+1] = num_img
for k, v in category_freq.items():
    category_freq[k] = v / num_images
# 2. For each category c, compute the category-level repeat factor:
#    r(c) = max(1, sqrt(t / f(c)))
category_rep = {
    int(cat_id): max(1.0, math.sqrt(repeat_thresh / cat_freq))
    for cat_id, cat_freq in category_freq.items()
}
# 3. For each image I, compute the image-level repeat factor:
#    r(I) = max_{c in I} r(c)
rep_factors = []
i = 0
for img in range(num_images):
    i +=1
    percent = 1.0 * i / num_images
    sys.stdout.write("%.4f"%percent)
    sys.stdout.write("\r")
    sys.stdout.flush()
    cat_ids = {int(coco_train.category_id_to_sorted_id[ann['category_id']]) for ann in coco_train[img][1]}
#     print(cat_ids)
    rep_factor = max({category_rep[cat_id] for cat_id in cat_ids})
    rep_factors.append(rep_factor)


In [14]:
save_path = '/mnt/data-disk2/xinting/project/dataset/LVIS/lvis_trainval_1230/lvis_trainval_1230_repeat_factor.npy'
np.save(save_path, rep_factors)

read_data = np.load(save_path)

In [17]:
sum(read_data)

64858.1885206132